In [ ]:
!pip install deeptime

import numpy as np
import math
import itertools
import sys
from deeptime.markov.msm import MarkovStateModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:00


#Tuples

The core datastructure in our implementation is a tuple. This captures the number of agents with a certain image score. For example, the tuple (2,3,1,0) states that there are 2 agents with an image score of 0, 3 with an image score of 1, 1 with an image score of 2, and none with an image score of 4.

#Tuple pairs

A tuple pair is used to represent the image scores associated with 2 separate strategies. E.g., ((2,3,1,0),(1,2,0,1)).

#Methods

We define several utility methods to create, and index tuples and tuple pairs. These depend on set lookups being non-deterministic (which should be the case on the same machine). Methods are as follows.
- `array_expand` creates all tuples from a single tuple whose first element is the number of agents in the system. E.g., `array_expand(set([tuple(5,0,0,0)]))` will create (5,0,0,0), (4,1,0,0), (4,0,1,0), ... (0,0,0,5).
- `make_indices(num_rep,num_s1,num_s2)` will create all possible tuple pairs given the number of reputations possible in the system `num_rep`, the number of agents allocated to strategy s1 (`num_s1`) and to strategy s2 (`num_s2`).
- `make_index(array_set)` given a set of tuple pairs in array_set, this will return two maps, one mapping integers to tuple pairs, and the other mapping tuple pairs to integers.

In [ ]:
def array_expand(array_set):
  """creates tuples with the possible combinations of tuples expecting array_set[0] to be the number of agents in the tuple. Example usage: `array_expand(set([tuple(5,0,0,0)]))`"""
  add=set()
  finished=False
  while not finished:
    finished=True
    for a in array_set:
      for j in range(len(a)-1):
        if a[j]!=0:
          b=list(a)
          b[j]-=1
          b[j+1]+=1
          b=tuple(b)
          if b not in add and b not in array_set:
            add.add(b)
            finished=False
    array_set.update(add)
  return array_set

def make_indices(num_rep,num_s1,num_s2):
  """This function splits agents to all possible allocations. E.g., make_indices(6,3,2) will allocate the 3 agents to all possible 6 reputation values in the first tuple and 2 agents to all possible reputation values in the second."""
  s1a=[0]*num_rep
  s2a=[0]*num_rep
  s1a[0]=num_s1
  s2a[0]=num_s2
  i1=array_expand(set([tuple(s1a)]))
  i2=array_expand(set([tuple(s2a)]))
  return itertools.product(i1,i2)


def make_index(array_set):
  """A simple caching function which allows one to map from ints to tuple pairs and back. returns d,r where d[i] takes an index and returns a tuple and r[a] takes a tuple and returns the index"""
  i=0
  d={}
  r={}
  for a in array_set:
    d[i]=a
    r[a]=i
    i+=1
  return (d,r)

#Modifying tuple pairs

The following two methods take in a tuple pair and an index in the pair (with the second tuple in the pair continuing the index).
- `increase_image` decreases the element in the index by 1, and increase the element following it by 1 (assuming the index is not at the end of the tuple).
- `decrease_image` decreases the element in the index by 1, and increases the preceeding element by 1 (assuming the index is not at the start  of the tuple).

In [ ]:
def increase_image(tup,image_index):
  """decreases the image at the current image_index, increases it at the next one, unless we're at maximum. Imageindex is an integer between 0 and 2*R-1, i.e., goes across the pair"""
  (n,m)=tup
  if image_index==len(n)-1 or image_index==2*len(n)-1:
    return tup
  n=list(n)
  m=list(m)
  if image_index<len(n):
    n[image_index]-=1
    n[image_index+1]+=1
  else:
    m[image_index-len(n)]-=1
    m[image_index-len(n)+1]+=1
  return (tuple(n),tuple(m))

def decrease_image(tup,image_index):
  """decreases the image at the current image_index, increases it at the next one, unless we're at maximum. Imageindex is an integer between 0 and 2*R-1, i.e., goes across the pair"""
  (n,m)=tup
  if image_index==0 or image_index==len(n):
    return tup
  n=list(n)
  m=list(m)
  if image_index<len(n):
    n[image_index]-=1
    n[image_index-1]+=1
  else:
    m[image_index-len(n)]-=1
    m[image_index-len(n)-1]+=1
  return (tuple(n),tuple(m))

# Eigenvectors

We seek to compute the dominant eigenvector of the transition matrix. We use the `deeptime` library for this.

In [ ]:
def powermethod(m):
  """Use the power method to compute the dominant eigenvector"""
  t=MarkovStateModel(m)
  return t.stationary_distribution
  #Uncomment the following to do this manually, but note that it is more numerically unstable than the code above.
  #v=np.random.rand(len(m))
  #v=v.astype(np.longdouble)
  #for i in range(iters):
  #  vp=np.matmul(m,v)
  #  vp/=np.sum(vp)
  #  (i,max(abs(vp-v))/min(v),max(abs(vp-v)),max(abs(vp-np.matmul(m,v))))
  #  nonzeromin=np.min(v[np.nonzero(v)])
  #  if max(abs(vp-v))/nonzeromin<=nonzeromin*1e-18:
  #    return vp
  #  v=vp
  #return v

# Donation probability

Consider a donor following a strategy $s_d$, deciding whether to donate to a recipient with and image score $i_r$. Fundamentally, **a donor donates if the image score is equal or greater than the strategy value**. Let
- $e_a$ be the _action noise_, the likelihood that even if the agent wanted to donate, they would accidentally not donate.
- $e_r$ be the _recall noise_, the likelihood that the donor would incorrectly estimate the recipient's image. If recall noise occurs, the percieved image is drawn uniformly from all possible image scores.
- $g$ be the system's _generosity_ value. This is the likelihood that a donor will donate even if they shouldn't.
- $r$ is the number of image scores possible.

The likelihood of donation is then computed as follows.

$$
p_{donate}=(1-e_a) \times
  \begin{cases}
  0 \mbox{ if } s_d > r \\
  (1-e_r) + e_r(1-\frac{r-s_d}{r} + g\frac{r-s_d}{r}) \mbox{ if } i_r \geq s_d \\
  g(1-e_r)+e_r(1-\frac{r-s_d}{r} + g\frac{r-s_d}{r}) \mbox{ otherwise}
\end{cases}
$$


In [ ]:
def prob_donate(s_d,i_r,e_a,e_r,g,r):
  if s_d>r:
    return 0
  x=(r-s_d)/r #small speedup: recalculate terms used multiple times, namely (r-s_d)/r
  if i_r>=s_d:
    return (1-e_a)*((1-e_r)+e_r*(1-x+x*g))
  else:
    return (1-e_a)*((1-e_r)*g+e_r*(1-x+x*g))

# Building the reputation transition matrix

Consider a system with only two strategies $s_1$, $s_2$, and an associated tuple pair $tup=(T_1,T_2)$. We can compute the likelihood of all new tuple pairs being generated following an interaction of these two strategies. To compute this likelihood, we must consider action and recall noise ($e_a, e_r$), generosity ($g$) and the number of image scores possible ($r$).

Assume a total of $N$ agents, that the donor follows strategy $s_d$ and has image $i_d$ and recipient follows strategy $s_r$ and has image $i_r$. Then the likelihood of these two agents interacting is
$$
p_{interact}=
\begin{cases}
  \frac{T_d[i_d]T_r[i_r]}{(N)(N-1)} \mbox{ if } s_d \neq s_r \\
  \frac{T_d[i_d]^2-T_d[i_d]}{(N)(N-1)} \mbox{ otherwise}
\end{cases}
$$

Here e.g., $T_d[i_d]$ denotes the number of agents at index $i_d$ in the tuple associated with strategy $s_d$.

We iterate over all possible tuples setting their probability to $p_{interact}p_{donate}$ if the image incremented, and to $p_{interact}(1-p_{donate})$ if the image is decremented.

The method `compute_transitions` takes in a tuple, strategies, noise and generosity parameters, and returns a map of new tuple $\to$ probabilities for non-zero likelihoods.

By calling this for all possible tuples, we can create the transition matrix. This is done by the `make_transition_matrix` which creates and indexes the tuples.

In [ ]:
def compute_transitions(tup,s1,s2,e_a,e_r,g):
  """takes in a ([n_1, ..., n_r],[m_1, ... m_r]) tuple and returns a [new_tuple]->likelihood tuple for those transitions"""
  probabilities={}

  (n,m)=tup
  num_agents=sum(n)+sum(m)
  for donor in range(len(n)+len(m)):
    for recipient in range(len(n)+len(m)):
      if (n+m)[donor]==0 or (n+m)[recipient]==0: #speedup: if there is no donor or recipient agent at this index, just continue.
        continue
      #donor and recipient are indices
      prob_play=0
      if donor!=recipient:
        prob_play=(n+m)[donor]*(n+m)[recipient]/(num_agents*(num_agents-1))
      else:
        prob_play=((n+m)[donor]**2-(n+m)[donor])/(num_agents*(num_agents-1))
      #we now have the likelihood of them playing against each other
      #now get the actual indices to work out the strategies.
      if donor-len(n)<0:
        strat1=s1
      else:
        strat1=s2
      if recipient-len(n)<0:
        strat2=s1
      else:
        strat2=s2
      image1=donor%len(n)
      image2=recipient%len(n)

      p_donate=prob_donate(strat1,image2,e_a,e_r,g,len(n))
      donate_tuple=increase_image(tup,donor)
      no_donate_tuple=decrease_image(tup,donor)

      probabilities[donate_tuple]=p_donate*prob_play+probabilities.get(donate_tuple,0)
      probabilities[no_donate_tuple]=(1-p_donate)*prob_play+probabilities.get(no_donate_tuple,0)
  return probabilities

def make_transition_matrix(num_s1,num_s2,s1,s2,e_a,e_r,g,r):
  tuples=list(make_indices(r,num_s1,num_s2))
  (index_to_tuple,tuple_to_index)=make_index(tuples)
  transition_matrix=np.zeros([len(tuples),len(tuples)])
  for t in tuples:
    transition_probs=compute_transitions(t,s1,s2,e_a,e_r,g)
    for p in transition_probs:
      transition_matrix[tuple_to_index[t],tuple_to_index[p]]=transition_probs[p]
  return transition_matrix

# Computing Expected Utilities

Given a tuple pair, associated strategies, noise and generosity parameters, we can compute the expected utility of each strategy by looking at their likelihood of interacting as a donor or recipient, the probability of a donation taking place, and the utility of donation or non-donation to them.


In [ ]:
def tuple_utility(tup,s1,s2,donate_util,recieve_util,e_a,e_r,g):
  """this returns a pair of utilities for each strategy by considering the likelihood that a pair of agents play each other and multiplying that by the likelihood of donating and the payoff for donating"""
  util={s1:0,s2:0}
  (n,m)=tup
  num_agents=sum(n)+sum(m)
  for donor in range(len(n)+len(m)):
    for recipient in range(len(n)+len(m)):
      if (n+m)[donor]==0 or (n+m)[recipient]==0: #speedup: if there is no donor or recipient agent at this index, just continue.
        continue
      #donor and recipient are indices
      prob_play=0
      if donor!=recipient:
        prob_play=(n+m)[donor]*(n+m)[recipient]/(num_agents*(num_agents-1))
      else:
        prob_play=((n+m)[donor]**2-(n+m)[donor])/(num_agents*(num_agents-1))
      #we now have the likelihood of them playing against each other
      #now get the actual indices to work out the strategies.
      if donor-len(n)<0:
        strat1=s1
      else:
        strat1=s2
      if recipient-len(n)<0:
        strat2=s1
      else:
        strat2=s2
      image1=donor%len(n)
      image2=recipient%len(n)

      p_donate=prob_donate(strat1,image2,e_a,e_r,g,len(n))
      util[strat1]+=p_donate*prob_play*donate_util
      util[strat2]+=p_donate*prob_play*recieve_util
  return util

We can also compute the utility of a group of agents where $|s_1|$ agents follow strategy $s_1$ and $|s_2|$ follow strategy $s_2$. For this, we need to compute the likelihood of each tuple arising, and multiply it by the utility computed above. The likelihood of the tuple arising is obtained by the vector of the stationary distribution of the transition matrix (computed via the power method).



In [ ]:
def compute_utility(num_s1,num_s2,s1,s2,e_a,e_r,g,r,donate_util,recieve_util):
  """computes the utility for a specific distribution of s1 and s2 strategies across all tuples with that number"""
  tm=make_transition_matrix(num_s1,num_s2,s1,s2,e_a,e_r,g,r)
  vec=powermethod(tm)
  util={s1:0,s2:0}
  tuples=list(make_indices(r,num_s1,num_s2))
  (index_to_tuple,tuple_to_index)=make_index(tuples)
  for i in range(len(vec)):
    t=index_to_tuple[i]
    u=tuple_utility(t,s1,s2,donate_util,recieve_util,e_a,e_r,g)
    util[s1]+=u[s1]*vec[i]
    util[s2]+=u[s2]*vec[i]
  return util

# From reputation to evolution dynamics

We assume that agents learn strategies on a different, and much slower, timescale than image scores get updated.

We treat strategy learning as a Fermi process. Here, a \emph{focal agent}, which may learn a new strategy, is picked from the population at random. If this agent follows a strategy $s_1$, it'll change it's strategy to $s_2$ with likelihood

$$\frac{1}{1+e^{-\beta(\pi_2-\pi_1)}}$$

where $\pi_2$ and $\pi_1$ are the expected utilities of strategy $s_2$ and $s_1$ respectively. The $\beta$ parameter controls the likelihood of imitation. A large $\beta$ makes adoption of a higher utility strategy more likely, while a small $\beta$ increases the likelihood of a random strategy being adopted.

We assume that our population of agents contains a single strategy for nearly all time periods, and is occassionaly invaded by one agent pursuing a different strategy. The _fixation probability_ computes the likelihood of this new strategy taking over the entire population as agents learn new behaviours. When a Fermi process is used for learning, the fixation probability of a strategy $s_2$ taking over from a strategy $s_1$ for a population of $N$ agents is computed as follows.

$$p_{fix}(s_1,s_2) = \frac{1}{1+ \sum_1^{N-1}\Pi_{k=1}^i e^{-\beta(\pi_2 - \pi_1)}}$$

We can define a matrix whose non-diagonal elements are $kp_{fix}(s_1,s_2)$ and whose diagonal elements (for strategy $s_1$) are $1-k\sum_s p_{fix}(s_1,s)$ where $k$ is a sufficiently small constant to ensure that all elements are positive. A simple value for $k$ is $1/|S|$ where $|S|$ is the number of strategies in the system.

In [ ]:
def strat_transition_matrix(num_agents,e_a,e_r,g,r,donate_util,recieve_util,beta):
  fm=np.zeros([r+1,r+1])
  for i in range(r+1):
    for j in range(r+1):
      s1=i
      s2=j
      sm=0
      for na in range(1,num_agents):
        tmp=1
        for k in range(1,na+1):
          u=compute_utility(k,num_agents-k,s1,s2,e_a,e_r,g,r,donate_util,recieve_util)
          us1=u[s1]
          us2=u[s2]

          tmp*=math.exp(-beta*(us2-us1)) #faster than calling fermi_learning, constant is the beta parameter in fermi
        sm+=tmp
      fixation=1/(1+sm)
      fm[i,j]=fixation
  fm/=r
  for i in range(r+1):
    fm[i,i]=1-(sum(fm[i])-fm[i,i])
  return fm

# Cooperation index

The stationary distribution of the matrix described above captures the fraction of time that each strategy exists alone in the system.

The _cooperation index_ of a system is a number between 0 and 1 denoting the fraction of cooperative actions which take place when a system is stable. We can compute this by considering the likelihood of being in some state (i.e., strategy), multiplied by the likelihood of a cooperative action (i.e., a donation) taking place. This latter likelihood in turn requires us to consider the likelihood of each image score tuple pair when agents follow that strategy, and the likelihood of donation in that case.

In [ ]:
def cooperation_index(num_agents,e_a,e_r,g,r,donate_util,recieve_util,beta):
  """The cooperation index is calculated as the likelihood of ending up in a single agent state (c.f., the strat_transition_matrix)
     times the likelihood of having some distribution of images in that state times the likelihood that an agent will donate in that state."""
  sum=0
  stm=strat_transition_matrix(num_agents,e_a,e_r,g,r,donate_util,recieve_util,beta)
  p=powermethod(stm)

  for i in range(len(p)):
    tm=make_transition_matrix(num_agents,0,i,0,e_a,e_r,g,r) #we can consider all agents being in the first tuple.
    vec=powermethod(tm)
    tuples=list(make_indices(r,num_agents,0))
    (index_to_tuple,tuple_to_index)=make_index(tuples)

    for j in range(len(vec)):
      current_tuple=index_to_tuple[j][0]
      probability_of_tuple=vec[j]

      for donor in range(len(current_tuple)):
        for recipient in range(len(current_tuple)):
          if current_tuple[donor]==0 or current_tuple[recipient]==0:
            continue
          prob_play=0
          if donor!=recipient:
            prob_play=current_tuple[donor]*current_tuple[recipient]/(num_agents*(num_agents-1))
          else:
            prob_play=(current_tuple[donor]**2-current_tuple[donor])/(num_agents*(num_agents-1))

          image1=donor
          image2=recipient

          p_donate=prob_donate(i,image2,e_a,e_r,g,r)
          sum+=p_donate*prob_play*probability_of_tuple*p[i]
  return sum

#Experiments
Below this point we move to generating results.

In [ ]:
np.set_printoptions(precision=8,suppress=True,threshold=sys.maxsize)

In [ ]:
import matplotlib.pyplot as plt
from collections import OrderedDict

linestyles_dict = OrderedDict(
    [('solid',               (0, ())),
     ('loosely dotted',      (0, (1, 10))),
     ('dotted',              (0, (1, 5))),
     ('densely dotted',      (0, (1, 1))),

     ('loosely dashed',      (0, (5, 10))),
     ('dashed',              (0, (5, 5))),
     ('densely dashed',      (0, (5, 1))),

     ('loosely dashdotted',  (0, (3, 10, 1, 10))),
     ('dashdotted',          (0, (3, 5, 1, 5))),
     ('densely dashdotted',  (0, (3, 1, 1, 1))),

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))])

colors_dict = OrderedDict(
    [('1', '#ff9933'),
     ('2', '#009999'),
     ('3', '#cc66ff'),
     ('4', '#6699ff'),
     ('5', '#ff9900'),
     ('6', '#009933')])

lines_dict = OrderedDict(
     [('1', (0, (1, 5))),
     ('2', (0, (5, 5))),
     ('3', (0, (3, 5, 1, 5))),
     ('4', (0, (1, 10))),
     ('5', (0, (5, 10))),
     ('6', (0, (3, 10, 1, 10, 1, 10))),
     ('7', (0, (1, 1))),
     ('8', (0, (5, 1))),
     ('9', (0, (3, 1, 1, 1)))])


In [ ]:
#figure 4a
fig=plt.figure()
plt.xlim(-0.02,0.22)
plt.ylim(-0.02,1.2)
plt.ylabel("Cooperation index")
plt.xlabel("Action and reputation noise")

xpoints = [0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2]

ypoints1=[]
ypoints2=[]
ypoints3=[]
for x in xpoints:
  ypoints1.append(cooperation_index(6,x,x,0,5,-0.1,1.0,10))
  ypoints2.append(cooperation_index(6,x,x,0.01,5,-0.1,1.0,10))
  ypoints3.append(cooperation_index(6,x,x,0.05,5,-0.1,1.0,10))

plt.plot(xpoints,ypoints1,color=colors_dict['1'], marker='o', linestyle=lines_dict['1'],
         label="g2=0")
plt.plot(xpoints,ypoints2,color=colors_dict['2'], marker='o', linestyle=lines_dict['2'],
         label="g2=0.01")
plt.plot(xpoints,ypoints3,color=colors_dict['3'], marker='o', linestyle=lines_dict['3'],
         label="g2=0.05")
plt.legend(loc='upper right')

In [ ]:
#figure 4b
fig=plt.figure()
plt.xlim(1.98,7.02)
plt.ylim(-0.02,1.2)
plt.ylabel("Cooperation index")
plt.xlabel("Num. agents")

xpoints = [2, 3, 4, 5, 6, 7]

ypoints1=[]
ypoints2=[]
ypoints3=[]
for x in xpoints:
  ypoints1.append(cooperation_index(x,0.025,0.025,0,5,-0.1,1.0,10))
  ypoints2.append(cooperation_index(x,0.025,0.025,0.01,5,-0.1,1.0,10))
  ypoints3.append(cooperation_index(x,0.025,0.025,0.05,5,-0.1,1.0,10))

plt.plot(xpoints,ypoints1,color=colors_dict['1'], marker='o', linestyle=lines_dict['1'],
         label="g2=0")
plt.plot(xpoints,ypoints2,color=colors_dict['2'], marker='o', linestyle=lines_dict['2'],
         label="g2=0.01")
plt.plot(xpoints,ypoints3,color=colors_dict['3'], marker='o', linestyle=lines_dict['3'],
         label="g2=0.05")
plt.legend(loc='upper right')

In [ ]:
#figure 4c num_agents,e_a,e_r,g,r,donate_util,recieve_util,beta
fig=plt.figure()
plt.xlim(1.98,8.02)
plt.ylim(-0.02,1.2)
plt.ylabel("Cooperation index")
plt.xlabel("Poss. image scores")

xpoints = [2, 3, 4, 5, 6, 7,8]

ypoints1=[]
ypoints2=[]
ypoints3=[]
for x in xpoints:
  ypoints1.append(cooperation_index(4,0.025,0.025,0,x,-0.1,1.0,10))
  ypoints2.append(cooperation_index(4,0.025,0.025,0.01,x,-0.1,1.0,10))
  ypoints3.append(cooperation_index(4,0.025,0.025,0.05,x,-0.1,1.0,10))

plt.plot(xpoints,ypoints1,color=colors_dict['1'], marker='o', linestyle=lines_dict['1'],
         label="g2=0")
plt.plot(xpoints,ypoints2,color=colors_dict['2'], marker='o', linestyle=lines_dict['2'],
         label="g2=0.01")
plt.plot(xpoints,ypoints3,color=colors_dict['3'], marker='o', linestyle=lines_dict['3'],
         label="g2=0.05")
plt.legend(loc='upper right')